In [3]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gensim.downloader
import numpy as np

from pathlib import Path

import glob

# 1. Чтение лемм

Дальнейшая разметка строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится

In [5]:
df_list = []

number_of_chunks = len(glob.glob("../research/lemmatized/*.csv"))
for i in range(number_of_chunks):
    df_list.append(
        pd.read_csv(f"../research/lemmatized/lemmatized_{i}.csv", sep=";")
    )

df = pd.concat(df_list, axis=0)
df

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
337142,dataset/Dataset/data/35185/table_5_meta.json,2,дата
337143,dataset/Dataset/data/35185/table_5_meta.json,3,дата
337144,dataset/Dataset/data/35185/table_5_meta.json,4,дата
337145,dataset/Dataset/data/35185/table_5_meta.json,5,дата


**TODO:** use unique only in furhter labeling + join with all lemmas

In [38]:
unique_col_name_df = df.drop_duplicates(subset=["column_name"])
unique_col_name_df

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
333591,dataset/Dataset/data/561562/table_2_meta.json,3,подъм
335746,dataset/Dataset/data/6446239/table_0_meta.json,5,секеи
336733,dataset/Dataset/data/956099/table_0_meta.json,0,ибаге
336734,dataset/Dataset/data/956099/table_0_meta.json,1,невадос


# 2. Чтение семантических типов

Некоторые типы были изменены, т.к. состояли из > 1 слова. Так как далее работаем с word2vec, эти типы были сокращены до одного слова.

In [6]:
labels = set()
with open("sem_types.txt", mode="r") as f:
    for line in f:
        labels.add(line.lower().strip())

labels = list(labels)
sem_types = dict((labels[i], i) for i in range(len(labels)))
sem_types

{'создатель': 0,
 'персона': 1,
 'полицейский': 2,
 'лингвист': 3,
 'группа': 4,
 'телешоу': 5,
 'турнир': 6,
 'плотина': 7,
 'день': 8,
 'идентификатор': 9,
 'устройство': 10,
 'остров': 11,
 'событие': 12,
 'элемент': 13,
 'авиакомпания': 14,
 'архив': 15,
 'сингл': 16,
 'газета': 17,
 'период': 18,
 'спортсмен': 19,
 'местонахождение': 20,
 'директор': 21,
 'мельница': 22,
 'телерадиокомпания': 23,
 'дизайнер': 24,
 'монарх': 25,
 'памятник': 26,
 'военная часть': 27,
 'ресторан': 28,
 'насекомое': 29,
 'владелец': 30,
 'конькобежец': 31,
 'растение': 32,
 'пещера': 33,
 'сообщество': 34,
 'казино': 35,
 'наездник': 36,
 'комик': 37,
 'экономист': 38,
 'тип': 39,
 'болезнь': 40,
 'самолет': 41,
 'страна': 42,
 'биомолекула': 43,
 'муниципалитет': 44,
 'демография': 45,
 'мода': 46,
 'скульптура': 47,
 'борец': 48,
 'договор': 49,
 'музей': 50,
 'гольфист': 51,
 'женщина': 52,
 'океан': 53,
 'молл': 54,
 'издатель': 55,
 'шифр': 56,
 'живопись': 57,
 'локомотив': 58,
 'робот': 59,
 '

# 3. Разметка

Каждому заголовку сопоставляется наиболее близкий семантический тип (метрика cosine similarity)

In [7]:
ru_model = gensim.downloader.load("word2vec-ruscorpora-300")

In [8]:
def get_most_similar(word: str) -> str|None:
    """Need to split afterwards because returns (label cos.sim.value)
    """

    max_similarity = 0
    max_similar_label = None
    
    try:
        ru_model.__getitem__([f"{word}_NOUN"])[0]
    except KeyError:
        return f"{max_similar_label} {max_similarity}"

    for label in sem_types.keys():
        try:
            similarity = ru_model.similarity(f"{label}_NOUN", f"{word}_NOUN")
        except KeyError:
            continue

        if similarity > max_similarity:
            max_similarity = similarity
            max_similar_label = label

    return f"{max_similar_label} {max_similarity}"

In [9]:
if not Path("./labelled/labelled.csv").exists():
    df_list = []
    for i in range(number_of_chunks):
        df_list.append(
            pd.read_csv(f"./lemmatized/lemma_{i}.csv", sep=";", on_bad_lines="warn")
        )
    df = pd.concat(df_list, axis=0)

    # df = df.sample(frac=0.001)
    
    df["label"] = df["column_name"].progress_apply(get_most_similar)
    df.to_csv("./labelled/labelled.csv", sep=";", index=False)
else:
    df = pd.read_csv("./labelled/labelled.csv", sep=";")

df

,table_id,column_id,column_name,label
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт,сингл 0.5395997762680054
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция,None 0
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда,команда 1.0000001192092896
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки,мотоциклист 0.3064345717430115
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год,год 0.9999999403953552
...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5_meta.json,2,дата,дата 1.0
3371474,dataset/Dataset/data/35185/table_5_meta.json,3,дата,дата 1.0
3371475,dataset/Dataset/data/35185/table_5_meta.json,4,дата,дата 1.0
3371476,dataset/Dataset/data/35185/table_5_meta.json,5,дата,дата 1.0


In [10]:
df[["label", "similarity"]] = df["label"].str.strip().str.split(" ", n=1, expand=True)
df

,table_id,column_id,column_name,label,similarity
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция,None,0
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда,команда,1.0000001192092896
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки,мотоциклист,0.3064345717430115
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год,год,0.9999999403953552
...,...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5_meta.json,2,дата,дата,1.0
3371474,dataset/Dataset/data/35185/table_5_meta.json,3,дата,дата,1.0
3371475,dataset/Dataset/data/35185/table_5_meta.json,4,дата,дата,1.0
3371476,dataset/Dataset/data/35185/table_5_meta.json,5,дата,дата,1.0


In [11]:
df["table_id"] = df["table_id"].str.strip().str.split("_meta.json", n=1, expand=True)[0]
df

,table_id,column_id,column_name,label,similarity
0,dataset/Dataset/data/7779522/table_0,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция,None,0
2,dataset/Dataset/data/2863377/table_11,1,команда,команда,1.0000001192092896
3,dataset/Dataset/data/2863377/table_11,9,очки,мотоциклист,0.3064345717430115
4,dataset/Dataset/data/4727026/table_0,0,год,год,0.9999999403953552
...,...,...,...,...,...
3371473,dataset/Dataset/data/35185/table_5,2,дата,дата,1.0
3371474,dataset/Dataset/data/35185/table_5,3,дата,дата,1.0
3371475,dataset/Dataset/data/35185/table_5,4,дата,дата,1.0
3371476,dataset/Dataset/data/35185/table_5,5,дата,дата,1.0


## Выбор порогового значения

|порог|кол-во заголовков, схожесть >= порог|
|-----|------------------------------------|
|0.1|2.993.652|
|0.2|2.993.550|
|0.3|2.968.070|
|0.4|2.754.081|
|0.5|2.186.338|
|0.6|1.659.870|
|0.7|1.505.095|
|0.8|1.435.068|
|0.9|1.427.649|
|1.0|901.335|

P.s. Значения >= 1.0 за счет погрешности чисел с палвающей точкой

In [12]:
for i in range(1, 10+1):
    sim = (df["similarity"].apply(float) >= i / 10).sum()
    print(f"{i / 10} : {sim}")

0.1 : 2993652
0.2 : 2993550
0.3 : 2968070
0.4 : 2754081
0.5 : 2186338
0.6 : 1659870
0.7 : 1505095
0.8 : 1435068
0.9 : 1427649
1.0 : 901335


In [13]:
threshold = 0.78

df_threshold = df[df["similarity"].astype(float) > threshold]
df_threshold.sort_values("similarity").head(10)

,table_id,column_id,column_name,label,similarity
3354967,dataset/Dataset/data/8757853/table_0,0,мыслитель,философ,0.7839148044586182
605035,dataset/Dataset/data/7846870/table_0,0,мыслитель,философ,0.7839148044586182
819200,dataset/Dataset/data/7847449/table_0,0,мыслитель,философ,0.7839148044586182
3163977,dataset/Dataset/data/628974/table_1,3,мотор,двигатель,0.7861505150794983
2640380,dataset/Dataset/data/3727914/table_16,3,мотор,двигатель,0.7861505150794983
2919969,dataset/Dataset/data/4273768/table_4,2,мотор,двигатель,0.7861505150794983
1413735,dataset/Dataset/data/760709/table_7,2,мотор,двигатель,0.7861505150794983
2108365,dataset/Dataset/data/3620310/table_11,3,мотор,двигатель,0.7861505150794983
2356856,dataset/Dataset/data/2377385/table_12,3,мотор,двигатель,0.7861505150794983
330778,dataset/Dataset/data/973893/table_3,2,мотор,двигатель,0.7861505150794983


In [14]:
df_threshold["label"].value_counts()[:20]

label
год           253209
название      189156
место         116915
дата          107649
команда        82552
примечание     61667
результат      58263
страна         42397
турнир         36697
клуб           24815
город          23303
категория      20853
пол            20066
население      19691
награда        19402
тип            18548
позиция        17792
спортсмен      17731
описание       15168
вес            13509
Name: count, dtype: int64

In [18]:
df_threshold.sort_values("table_id", inplace=True)
df_threshold

/tmp/ipykernel_17741/420617415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threshold.sort_values("table_id", inplace=True)


,table_id,column_id,column_name,label,similarity
334870,dataset/Dataset/data/1000006/table_0,4,примечание,примечание,1.0000001192092896
334867,dataset/Dataset/data/1000006/table_0,1,спектакль,пьеса,0.7898139357566833
334866,dataset/Dataset/data/1000006/table_0,0,театр,театр,1.0
1459759,dataset/Dataset/data/1000006/table_1,2,название,название,1.0
1459758,dataset/Dataset/data/1000006/table_1,0,год,год,0.9999999403953552
...,...,...,...,...,...
1252677,dataset/Dataset/data/999987/table_0,7,место,место,0.9999999403953552
1252673,dataset/Dataset/data/999987/table_0,0,год,год,0.9999999403953552
1252676,dataset/Dataset/data/999987/table_0,6,результат,результат,1.0
1252674,dataset/Dataset/data/999987/table_0,1,город,город,1.0000001192092896


In [19]:
df_threshold.to_csv(f"./labelled/labelled_threshold_{threshold}.csv", sep=";", index=False)